In [1]:
## Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn import svm
from sklearn.metrics import *
import zipfile
from zipfile import ZipFile
from io import BytesIO
import sys
import glob
import datetime
import time
import boto.s3
from boto.s3.key import Key 
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('diabetes1.csv')

In [3]:
col_list=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train[col_list]
y_train=df_train['Outcome']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test[col_list]
y_test=df_test['Outcome']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)
print(x_train.shape,y_train.shape)

(417, 8) (417,)


In [4]:
accuracy =[]
model_name =[]
dataset=[]
f1score = []
precision = []
recall = []
true_positive =[]
false_positive =[]
true_negative =[]
false_negative =[]

In [5]:
logreg = LogisticRegression()
## fitiing the model
logreg.fit(x_train_sc, y_train)
filename = 'logReg_model.sav'
pickle.dump(logreg,open(filename,'wb'))
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
prediction = logreg.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
dataset.append('Training')
model_name.append('Logistic Regression')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[193,  38],
       [ 62, 124]], dtype=int64)

In [7]:
feature_list = list(x_train.columns)
importances = list(logreg.coef_)
print(importances)
print(x_train.columns)

[array([ 0.4008114 ,  1.05132873, -0.24015178,  0.00768797,  0.04674092,
        0.82749388,  0.31507642,  0.18746321])]
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


In [8]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[231,   0],
       [  0, 186]], dtype=int64)

In [10]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Glucose              Importance: 0.23
Variable: BMI                  Importance: 0.19
Variable: Age                  Importance: 0.14
Variable: DiabetesPedigreeFunction Importance: 0.12
Variable: Pregnancies          Importance: 0.08
Variable: BloodPressure        Importance: 0.08
Variable: SkinThickness        Importance: 0.08
Variable: Insulin              Importance: 0.08


In [13]:
col_list=['Pregnancies','Glucose','BloodPressure','SkinThickness','BMI','DiabetesPedigreeFunction','Age']
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train[col_list]
y_train=df_train['Outcome']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test[col_list]
y_test=df_test['Outcome']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)
print(x_train.shape,y_train.shape)

(417, 7) (417,)


In [14]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[231,   0],
       [  0, 186]], dtype=int64)

In [16]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Glucose              Importance: 0.28
Variable: BMI                  Importance: 0.19
Variable: Age                  Importance: 0.15
Variable: DiabetesPedigreeFunction Importance: 0.12
Variable: BloodPressure        Importance: 0.09
Variable: SkinThickness        Importance: 0.09
Variable: Pregnancies          Importance: 0.08


Conclusion : We have found after feature selection that these 7 columns have more importance and it's better to make the model with these.